### Importing Modules

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import yfinance as yf

### Setting up the credentials for google sheets api locally and calling it to get access to the sheets

In [ ]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
path = r"XXXXXX" # Replace XXXXX to the place where the google sheets api (creds.json) is kept 
creds = ServiceAccountCredentials.from_json_keyfile_name(path + "creds.json",  scope, token_uri = "https://oauth2.googleapis.com/token")
client = gspread.authorize(creds)

### Accessing and updating the sheets with data of each stock from yahoo finance

In [ ]:
sheet = client.open(r'XXXXXX') # Opening the google sheet. Replace XXXXXX with the google sheet name
sheet1 = sheet.worksheet(r"XXXXXX") # Calling the worksheet. Replace XXXXXX with the name of the worksheet in the google sheet

maxlen = 300  # Max number of rows to iterate through

for i in range(0,maxlen):  # Looping through all the rows in the work sheet
    t = sheet1.cell(i+2,4).value  # This is the cell in 4th column where the ticker for the stock is located.
    if not len(t)==0: # If the ticker is non-null, then attempt the following
        try:
            s = yf.Ticker(t) # Calling the yahoo finance object using the ticker obtained in the above step
            sector = s.info.get('sector') # Obtain the sector of the stock
            industry = s.info.get('industry') # Obtain the industry of the stock
            sheet1.update_cell(i+2,11,sector) # Update the 11th column in the google sheet with the sector name
            sheet1.update_cell(i+2,12,industry) # Update the 12th column in the google sheet with the industry name
            df = s.history(period="2y") # Create the dataframe of all the data for the last two years of data
            p = df.tail(1)['Close'][0] # Get the most current stock price by calling the last row of the dataframe
            sheet1.update_cell(i+2,6, p) # Update the 6th column in the google sheet with the current price
            q = list(df.loc['2020-02-14':'2020-02-14']['Close'])[0] # Get the stock price on 14th Feb 2020
            sheet1.update_cell(i+2,7, q) # Update the 7th column in the google sheet with the price from 14th Feb 2020
            r = list(df.loc['2019-09-16':'2019-09-16']['Close'])[0] # Get the stock price on 16th sep 2019
            sheet1.update_cell(i+2,9, r) # Update the 9th column in the google sheet with the price from 16th Sep 2019
        except:
            pass
    else:
        pass